### In this exercise, I will perform DNN based on dataset CIFAR10

#### a) Create a deep network consisting of 20 hidden layers containing 100 neurons each (there are too many, but that's the moral of the exercise). Use the He initialisation and the ELU activation function.

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt

In [7]:
keras.backend.clear_session()
tf.random.set_seed(0)
np.random.seed(0)

network = keras.models.Sequential()
network.add(keras.layers.Flatten(input_shape = [32,32,3]))

for _ in range(20):
    network.add(keras.layers.Dense(100, 
                                  kernel_initializer="he_normal", # these parameters are default values
                                  activation="elu"))

#### b) Exercise: Using Nadam optimization and early stopping, train the network on the CIFAR10 dataset. You can load it with keras.datasets.cifar10.load_data(). The dataset is composed of 60,000 32 × 32–pixel color images (50,000 for training, 10,000 for testing) with 10 classes, so you'll need a softmax output layer with 10 neurons. Remember to search for the right learning rate each time you change the model's architecture or hyperparameters.

In [9]:
(X_train_full, y_train_full), (X_test_full, y_test_full) = keras.datasets.cifar10.load_data()

In [12]:
X_train_full.shape, X_test_full.shape

((50000, 32, 32, 3), (10000, 32, 32, 3))

##### As validation dataset, I will use 5000 observations of each training and test dataset.

In [16]:
X_train = X_train_full[5000:]
y_train = y_train_full[5000:]
X_valid = X_train_full[:5000]
y_valid = y_train_full[:5000]

Now, I will build my optimizer.

In [17]:
network.add(keras.layers.Dense(10, activation="softmax"))
optimizer = keras.optimizers.Nadam(learning_rate=0.001)
network.compile(loss = 'sparse_categorical_crossentropy',
                optimizer = optimizer,
                metrics = ["accuracy"])


early_stopping_cb = keras.callbacks.EarlyStopping(patience=10) # Because of early stopping, I have created validation dataset
checkpoint_cb = keras.callbacks.ModelCheckpoint("GNN_exercise_2.h5", save_best_only=True)
callbacks = [early_stopping_cb, checkpoint_cb]

network.fit(X_train, y_train, epochs = 50,
            validation_data = [X_valid, y_valid],
            callbacks = callbacks)

network = keras.models.load_model("GNN_exercise_2.h5")
network.evaluate(X_valid, y_valid)


Epoch 1/50
1407/1407 [==============================] - 28s 16ms/step - loss: 2.9553 - accuracy: 0.2145 - val_loss: 2.1783 - val_accuracy: 0.2208
Epoch 2/50
1407/1407 [==============================] - 19s 13ms/step - loss: 1.9279 - accuracy: 0.2888 - val_loss: 1.9115 - val_accuracy: 0.2828
Epoch 3/50
1407/1407 [==============================] - 19s 13ms/step - loss: 1.8716 - accuracy: 0.3127 - val_loss: 1.9317 - val_accuracy: 0.2900
Epoch 4/50
1407/1407 [==============================] - 20s 14ms/step - loss: 1.8283 - accuracy: 0.3306 - val_loss: 1.9108 - val_accuracy: 0.2864
Epoch 5/50
1407/1407 [==============================] - 23s 16ms/step - loss: 1.8128 - accuracy: 0.3384 - val_loss: 1.9881 - val_accuracy: 0.3264
Epoch 6/50
1407/1407 [==============================] - 25s 18ms/step - loss: 1.7759 - accuracy: 0.3563 - val_loss: 1.8421 - val_accuracy: 0.3458
Epoch 7/50
1407/1407 [==============================] - 27s 19ms/step - loss: 1.7508 - accuracy: 0.3672 - val_loss: 1.8397 -

[1.6971863508224487, 0.38339999318122864]

#### c) Now try adding Batch Normalization and compare the learning curves: Is it converging faster than before? Does it produce a better model? How does it affect training speed?

In [19]:
keras.backend.clear_session()
tf.random.set_seed(0)
np.random.seed(0)

network_batched = keras.models.Sequential()
network_batched.add(keras.layers.Flatten(input_shape = [32,32,3]))
network_batched.add(keras.layers.BatchNormalization())

for _ in range(20):
    network_batched.add(keras.layers.Dense(100, kernel_initializer="he_normal"))
    network_batched.add(keras.layers.BatchNormalization())
    network_batched.add(keras.layers.Activation("elu"))
network_batched.add(keras.layers.Dense(10, activation="softmax"))

optimizer = keras.optimizers.Nadam(learning_rate=0.001)
network_batched.compile(loss = 'sparse_categorical_crossentropy',
                optimizer = optimizer,
                metrics = ["accuracy"])


early_stopping_cb = keras.callbacks.EarlyStopping(patience=10) # Because of early stopping, I have created validation dataset
checkpoint_cb = keras.callbacks.ModelCheckpoint("GNN_exercise_2_1.h5", save_best_only=True)
callbacks = [early_stopping_cb, checkpoint_cb]

network_batched.fit(X_train, y_train, epochs = 50,
            validation_data = [X_valid, y_valid],
            callbacks = callbacks)

network_batched = keras.models.load_model("GNN_exercise_2_1.h5")
network_batched.evaluate(X_valid, y_valid)

Epoch 1/50
1407/1407 [==============================] - 46s 23ms/step - loss: 1.8465 - accuracy: 0.3364 - val_loss: 1.6822 - val_accuracy: 0.3910
Epoch 2/50
1407/1407 [==============================] - 32s 23ms/step - loss: 1.6903 - accuracy: 0.4000 - val_loss: 1.6440 - val_accuracy: 0.4064
Epoch 3/50
1407/1407 [==============================] - 32s 23ms/step - loss: 1.6234 - accuracy: 0.4221 - val_loss: 1.5868 - val_accuracy: 0.4316
Epoch 4/50
1407/1407 [==============================] - 32s 23ms/step - loss: 1.5627 - accuracy: 0.4450 - val_loss: 1.4923 - val_accuracy: 0.4600
Epoch 5/50
1407/1407 [==============================] - 32s 22ms/step - loss: 1.5203 - accuracy: 0.4589 - val_loss: 1.5501 - val_accuracy: 0.4456
Epoch 6/50
1407/1407 [==============================] - 32s 23ms/step - loss: 1.4868 - accuracy: 0.4744 - val_loss: 1.4727 - val_accuracy: 0.4698
Epoch 7/50
1407/1407 [==============================] - 32s 22ms/step - loss: 1.4473 - accuracy: 0.4866 - val_loss: 1.4517 -

[1.299830675125122, 0.545199990272522]